In [137]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from anytree import Node

%matplotlib inline

# Read in Data

In [503]:
data_dir = '../data/'

# default data
train_values = pd.read_csv(data_dir+'train_values.csv')
train_labels = pd.read_csv(data_dir+'train_labels.csv')
test_values = pd.read_csv(data_dir+'test_values.csv')

train_values = train_values.drop(columns=['patient_id'])
test_values = test_values.drop(columns=['patient_id'])

npatients = len(train_values)

# label each feature as numerical or categorical
num_or_cat = [1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0]

# data where categoricals are converted to binary (credit Nora)

def convert_columns(df, cols=[]):
    for col in cols:
        df[col] = df[col].astype(str)
    return df

process_values = convert_columns(process_values, cols=['chest_pain_type', 'resting_ekg_results'])
process_test = convert_columns(process_test, cols=['chest_pain_type', 'resting_ekg_results'])

train_values_binary = pd.get_dummies(process_values)
test_values_binary = pd.get_dummies(process_test)

num_or_cat_binary = [1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# Entropy and Laplace-corrected probability functions

In [507]:
def calc_entropy(labels):
    
    n = len(labels)
    n_yes = labels.sum()
    frac_yes = 1.0 * n_yes / n
    frac_no = 1.0 - frac_yes
    if (n_yes == n) | (n_yes == 0):
        entropy = 0
    else:
        entropy = - frac_yes * np.log2(frac_yes) - frac_no * np.log2(frac_no)
    
    return entropy

def calc_laplace_prob(labels):
    
    n = len(labels)
    n_yes = labels.sum()
    prob = (n_yes + 1.0) / (n + 2.0)
    
    return prob

In [530]:
parent_entropy = calc_entropy(train_labels['heart_disease_present'])
parent_prob = calc_laplace_prob(train_labels['heart_disease_present'])
print("Parent size (number of patients): " + str(npatients))
print("Parent entropy: " + str(parent_entropy))
print("Parent prob: " + str(parent_prob))

Parent size (number of patients): 180
Parent entropy: 0.9910760598382222
Parent prob: 0.44505494505494503


In [475]:
train_values

,patient_id,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
0,0z64un,1,normal,128,2,0,0,2,308,0.0,1,45,170,0
1,ryoo3j,2,normal,110,3,0,0,0,214,1.6,0,54,158,0
2,yt1s1x,1,normal,125,4,3,0,2,304,0.0,1,77,162,1
3,l2xjde,1,reversible_defect,152,4,0,0,0,223,0.0,1,40,181,0
4,oyt4ek,3,reversible_defect,178,1,0,0,2,270,4.2,1,59,145,0
5,ldukkw,1,normal,130,3,0,0,0,180,0.0,1,42,150,0
6,2gbyh9,2,reversible_defect,150,4,2,0,2,258,2.6,0,60,157,0
7,daa9kp,2,fixed_defect,150,4,1,0,2,276,0.6,1,57,112,1
8,3nwy2n,3,reversible_defect,170,4,0,0,2,326,3.4,1,59,140,1
9,1r508r,2,normal,120,3,0,0,0,219,1.6,0,50,158,0


# Information gain function for splitting a sample into subsets (Categorical feature)

In [510]:
def information_gain(values, labels):
    
    unique_vals = values.unique()
    
    n = len(labels)
    entropy_parent = calc_entropy(labels)
    
    sum_entropy = 0
    
    for v in unique_vals:
        
        idx = values==v
        subset = values[idx]
        labels_subset = labels[idx]
        size_subset = len(subset)
        entropy = calc_entropy(labels_subset)
        
        # Weight each subset by size of subset
        sum_entropy += (1.0 * size_subset / n) * entropy
        
    return entropy_parent - sum_entropy

# Information gain when splitting on each categorical feature (Thalium is best)

In [511]:
print(information_gain(train_values['thal'],  train_labels['heart_disease_present']))
print(information_gain(train_values['sex'], train_labels['heart_disease_present']))
print(information_gain(train_values['resting_ekg_results'], train_labels['heart_disease_present']))
print(information_gain(train_values['chest_pain_type'], train_labels['heart_disease_present']))
print(information_gain(train_values['num_major_vessels'], train_labels['heart_disease_present']))
print(information_gain(train_values['slope_of_peak_exercise_st_segment'], train_labels['heart_disease_present']))
print(information_gain(train_values['fasting_blood_sugar_gt_120_mg_per_dl'], train_labels['heart_disease_present']))
print(information_gain(train_values['exercise_induced_angina'], train_labels['heart_disease_present']))

0.22012798678465229
0.08617545305042906
0.022056499408533048
0.199416882884353
0.14050285839657706
0.0988136740991179
8.232688968323743e-06
0.1497987086300061


# Information gain function for splitting a sample into subsets (Numerical feature)

In [512]:
def information_gain_float(values, labels):
    
    n = len(labels)
    entropy_parent = calc_entropy(labels)
    
    sorted_vals = values.sort_values()
    unique_vals = sorted_vals.unique()
    
    num_sorted_vals = len(unique_vals)-1
    if num_sorted_vals==0:
        return 0.0, unique_vals
    
    sum_entropy = np.zeros(num_sorted_vals)
    tally = 0
    
    # Loop over all possible split values
    for v in unique_vals[:-1]:
        
        for i in range(0, 2):
            
            if i == 0:
                idx = values<=v
            if i == 1:
                idx = values>v
            subset = values[idx]
            labels_subset = labels[idx]
            size_subset = len(subset)
            entropy = calc_entropy(labels_subset)
        
            # weight entropy of subset by size of subset
            sum_entropy[tally] += (1.0 * size_subset / n) * entropy
        
        tally += 1
        
    # Choose best split value
    sum_entropy_min = np.amin(sum_entropy)

    # return information gain and best split value
    return entropy_parent - sum_entropy_min, unique_vals[np.argmin(sum_entropy)]

# Information gain and split value when splitting on each numerical feature (Oldpeak is best)

In [513]:
print(information_gain_float(train_values['age'],  train_labels['heart_disease_present']))
print(information_gain_float(train_values['resting_blood_pressure'],  train_labels['heart_disease_present']))
print(information_gain_float(train_values['serum_cholesterol_mg_per_dl'],  train_labels['heart_disease_present']))
print(information_gain_float(train_values['oldpeak_eq_st_depression'],  train_labels['heart_disease_present']))
print(information_gain_float(train_values['max_heart_rate_achieved'],  train_labels['heart_disease_present']))

(0.04611604755974963, 54)
(0.014396269869434097, 108)
(0.020988066108446057, 271)
(0.1014694857425571, 0.7)
(0.08691940223770644, 147)


# Recursive decision tree classifier

In [518]:
def decision_tree(values, labels, num_or_cat, parentNode, lvl=0, max_tree_depth=100, min_split_size=5):
    
    # max tree depth = one way to control size of tree
    if lvl>max_tree_depth:
        return parentNode
    
    inf_gains = np.zeros(values.shape[1])
    split_points = np.zeros(values.shape[1])
    
    # loop over features, find best information gain
    for i,column in enumerate(values):
        
        if num_or_cat[i]:
            
            inf_gains[i], split_points[i] = information_gain_float(values[column], labels)
            
        else:
            
            inf_gains[i] = information_gain(values[column], labels)
            
    idx_max = np.argmax(inf_gains)
    col_max = values.iloc[:,idx_max]
    
    # for numerical features
    if num_or_cat[idx_max]:
        
        # split parent into subsets based on best feature
        idx_nset = np.where(values.iloc[:,idx_max]<=split_points[idx_max])[0]
        newset = values.iloc[idx_nset,:]
        newlab = labels.iloc[idx_nset]
        purity = (np.sum(newlab) == len(newset)) | (np.sum(newlab) == 0)
        lprob =  calc_laplace_prob(newlab)
        title = values.columns[idx_max] + " <= " + str(split_points[idx_max])
        
        # create leaf
        tmpNode = Node(title, imax=idx_max, val=split_points[idx_max], cond='leq', prob=lprob, size=len(newset), pure=purity, level=lvl+1, parent=parentNode)
        
        if (tmpNode.size>min_split_size) & (not tmpNode.pure):
            
            tmpNode = decision_tree(newset, newlab, num_or_cat, tmpNode, lvl=lvl+1)
        
        idx_nset = np.where(values.iloc[:,idx_max]>split_points[idx_max])[0]
        newset = values.iloc[idx_nset,:]
        newlab = labels.iloc[idx_nset]
        purity = (np.sum(newlab) == len(newset)) | (np.sum(newlab) == 0)
        lprob = calc_laplace_prob(newlab)
        title = values.columns[idx_max] + " > " + str(split_points[idx_max])
        
        # create leaf
        tmpNode = Node(title, imax=idx_max, val=split_points[idx_max], cond='g', prob=lprob, size=len(newset), pure=purity, level=lvl+1, parent=parentNode)
        
        # keep going w/ recursion if not reached minimum split size
        if (tmpNode.size>min_split_size) & (not tmpNode.pure):
        
            tmpNode = decision_tree(newset, newlab, num_or_cat, tmpNode, lvl=lvl+1)
        
    # if categorical feature
    else:
        
        unique_vals = col_max.unique()
        for uval in unique_vals:
            
            idx_nset = np.where(values.iloc[:,idx_max]==uval)[0]
            newset = values.iloc[idx_nset,:]
            newlab = labels.iloc[idx_nset]
            lprob = (np.sum(newlab) + 1.0) / (len(newlab) + 2.0)
            title = values.columns[idx_max] + ' = ' + str(uval)
            purity = (np.sum(newlab) == len(newset)) | (np.sum(newlab) == 0)
            
            # create leaf
            tmpNode = Node(title, imax=idx_max, val=uval, cond='eq', prob=lprob, size=len(newset), pure=purity, level=lvl+1, parent=parentNode)
            
            # keep going w/ recursion if not reached minimum split size
            if (tmpNode.size>min_split_size) & (not tmpNode.pure):
                
                tmpNode = decision_tree(newset, newlab, num_or_cat, tmpNode, lvl=lvl+1)
            
    return parentNode

# Train decision tree using data

In [520]:
topNode = Node("Top", prob=parent_prob, size=npatients, pure=False, level=0)

tree = decision_tree(train_values, train_labels['heart_disease_present'], num_or_cat, topNode)

# Bad tree printing

In [526]:
for child in tree.children:
    
    print(child.name)
    print(child.val)
    print(child.size)
    print(child.pure)
    print(child.prob)
    
#     for ch in child.children:
        
#         print("imax: " + str(ch.imax) + " level 2")
#         print(ch.val)
#         print(ch.size)
#         print(ch.pure)
#        print(ch.prob)
        
#         for c in ch.children:
            
#             print("imax: " + str(c.imax) + " level 3")
#             print(c.val)
#             print(c.size)
#             print(c.pure)
#             print(c.prob)
            
#             for d in c.children:
            
#                 print("imax: " + str(d.imax) + " level 4")
#                 print(d.val)
#                 print(d.size)
#                 print(d.pure)
#                 print(d.prob)
        


thal = normal
normal
98
False
0.21
thal = reversible_defect
reversible_defect
74
False
0.75
thal = fixed_defect
fixed_defect
8
False
0.5


# Auxillary tree methods

In [527]:
# get all childless leaves
def get_leaves(tree):
    
    leaves = []
    
    def _get_leaves(node):
        
        if node is not None:
            
            if len(node.children) == 0:
                
                leaves.append(node)
                
            else:
                
                for child in node.children:
                    
                    _get_leaves(child)
    
    _get_leaves(tree)
    
    return leaves

# string together conditional statements going up from leaf to parent node
def get_conditionals(vals, node):
        
    if node.cond == 'eq':
        tidx = vals.iloc[:,node.imax]==node.val
    elif node.cond == 'leq':
        tidx = vals.iloc[:,node.imax]<=node.val
    elif node.cond == 'g':
        tidx = vals.iloc[:,node.imax]>node.val

    if node.level > 1:
        idx = tidx & get_conditionals(vals, node.parent)
    else:
        idx = tidx
    return idx

# Predict heart disease probabilities using trained decision tree

In [529]:
children = get_leaves(tree)
probs = np.zeros(len(test_values))
for child in children:
    tidx = get_conditionals(test_values, child)
    probs[tidx] = child.prob
    
print(probs)

[0.33333333 0.07142857 0.97368421 0.04347826 0.85714286 0.04347826
 0.33333333 0.83333333 0.07142857 0.75       0.04347826 0.57142857
 0.07142857 0.97368421 0.04347826 0.04347826 0.04347826 0.08333333
 0.83333333 0.04347826 0.97368421 0.07142857 0.04347826 0.33333333
 0.88888889 0.83333333 0.57142857 0.04347826 0.2        0.04347826
 0.97368421 0.33333333 0.83333333 0.16666667 0.5        0.07142857
 0.5        0.6        0.57142857 0.04347826 0.97368421 0.16666667
 0.97368421 0.25       0.97368421 0.04347826 0.07142857 0.66666667
 0.04347826 0.2        0.07142857 0.08333333 0.97368421 0.75
 0.57142857 0.04347826 0.83333333 0.08333333 0.04347826 0.16666667
 0.66666667 0.97368421 0.07142857 0.85714286 0.04347826 0.85714286
 0.85714286 0.33333333 0.97368421 0.85714286 0.08333333 0.97368421
 0.97368421 0.97368421 0.97368421 0.97368421 0.97368421 0.83333333
 0.85714286 0.57142857 0.97368421 0.04347826 0.85714286 0.97368421
 0.66666667 0.04347826 0.16666667 0.33333333 0.07142857 0.5       ]


# Prep prediction for submission (credit Nora)

In [495]:
temp_values = pd.read_csv(data_dir+'test_values.csv')
test_patient_id = temp_values['patient_id'].as_matrix()
sub = pd.DataFrame(np.vstack([test_patient_id, probs]).T)
sub.to_csv('submission_oct9.csv', sep=',', header=['patient_id','heart_disease_present'], index=False)

/home/andrew/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
